In [1]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
from PIL import Image
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import random
import tensorflow as tf
import pickle

Using TensorFlow backend.


In [2]:
def load_cfar10_batch(cifar10_dataset_folder_path, batch_id):
    with open(cifar10_dataset_folder_path + '/data_batch_' + str(batch_id), mode='rb') as file:
        # note the encoding type is 'latin1'
        batch = pickle.load(file, encoding='latin1')
        
    features = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
    labels = batch['labels']
        
    return features, labels

In [3]:
def scaleImage(inputMatrix):
    #vecShape = np.shape(inputMatrix)
    #print(vecShape)
    #vecShape(1) = 400
    #vecShape(2) = 400
    retMatrix = np.zeros([500, 400, 400, 3])
    for num in range(np.shape(retMatrix)[0]):
        img = Image.fromarray(inputMatrix[num,:,:,:], 'RGB')

        #resizedImage = img.resize([400,400], resample=0, box=None)
        #resizedImage.size
        retMatrix[num,:,:,:] = np.array(img.resize([400,400], resample=0, box=None))
        #np.shape(pix)
    return retMatrix

In [4]:
# Get images
pictures, labels = load_cfar10_batch('C:/Users/adith/Documents/MLAB CAR/CIFAR_data/cifar-10-batches-py', 1)
resizePictures = scaleImage(pictures)
image = resizePictures
# img = load_img('human.png').resize([400,400], resample=0, box=None)
# image = img_to_array(img)
# image = np.array(image, dtype=float)
# img

In [5]:
X = rgb2lab(1.0/255*image)[:,:,:,0]
Y = rgb2lab(1.0/255*image)[:,:,:,1:]
Y /= 128
X = X.reshape(500, 400, 400, 1)
Y = Y.reshape(500, 400, 400, 2)

In [7]:
# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [8]:
# Finish model
model.compile(optimizer='rmsprop', loss='mse')

In [9]:
model.fit(x=X, y=Y, batch_size=5, epochs=40)

Epoch 1/40
500/500 [==============================] - 173s 346ms/step - loss: 0.0223
Epoch 2/40
500/500 [==============================] - 198s 396ms/step - loss: 0.0114
Epoch 3/40
500/500 [==============================] - 215s 429ms/step - loss: 0.0113
Epoch 4/40
500/500 [==============================] - 163s 327ms/step - loss: 0.0113
Epoch 5/40
500/500 [==============================] - 181s 361ms/step - loss: 0.0113
Epoch 6/40
500/500 [==============================] - 218s 436ms/step - loss: 0.0113
Epoch 7/40
500/500 [==============================] - 228s 457ms/step - loss: 0.0112
Epoch 8/40
500/500 [==============================] - 219s 438ms/step - loss: 0.0113
Epoch 9/40
500/500 [==============================] - 210s 421ms/step - loss: 0.0112
Epoch 10/40
500/500 [==============================] - 193s 387ms/step - loss: 0.0116
Epoch 11/40
500/500 [==============================] - 175s 350ms/step - loss: 0.0113
Epoch 12/40
500/500 [==============================] - 174s 348

In [15]:
img = load_img('human.png').resize([400,400], resample=0, box=None)
imageToTest = img_to_array(img)
imageToTest = np.array(imageToTest, dtype=float)
XTest = rgb2lab(1.0/255*imageToTest)[:,:,0]
YTest = rgb2lab(1.0/255*imageToTest)[:,:,1:]
YTest /= 128
np.shape(YTest)


(400, 400, 2)

In [16]:
XTest = XTest.reshape(1, 400, 400, 1)
YTest = YTest.reshape(1, 400, 400, 2)

In [20]:
print(model.evaluate(XTest, YTest, batch_size=1))
output = model.predict(XTest)
output *= 128
# Output colorizations
cur = np.zeros((400, 400, 3))
cur[:,:,0] = XTest[0][:,:,0]
cur[:,:,1:] = output[0]
cur = lab2rgb(cur)
imsave("recolor_11_25_18_18_23.png", cur)
imsave("gray_version_11_25_18_18_23.png", rgb2gray(cur))

1/1 [==============================] - 0s 115ms/step
0.017200805246829987


In [22]:


cur = lab2rgb(cur)
im = Image.fromarray(np.uint8(cm.gist_earth(cur)*255))
im

NameError: name 'Image' is not defined